<h1>Getting Started</h1>

In [29]:
# if you are running in the notebooks folder example cloned from github this line must be included
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

<h3>Import CtllDes, the core and utils elements </h3>

In [30]:
import numpy as np
import CtllDes 
from CtllDes.core import ctll, satellite

## Building an orbit 
#### The utility function <strong>helio_geo_synchronous</strong> receives an integer, and returns a dataframe with all the possible circular orbit classic elements that are helio and geo synchronous with a D days period. It is pretty straigthforward to transform the desired filtered row into a classical State.

In [31]:
from CtllDes.utils import sscalc as sc

D = 7
orb_df = sc.helio_rgt_synchronous(D)
orb_df

,radius,inc,N
0,11544.182918,142.248541,7.000000
1,11389.626716,138.961424,7.142857
2,11240.148578,136.071863,7.285714
3,11095.486092,133.496401,7.428571
4,10955.395204,131.177130,7.571429
...,...,...,...
66,6535.898060,96.204172,16.428571
67,6498.279346,96.079672,16.571429
68,6461.197311,95.958727,16.714286
69,6424.639809,95.841203,16.857143


##### Lets say that you want to filter out heights. One easy way to filter out whatever state you may want to generalize uses pandas loc. If you are not familiar with pandas, read the basic examples in their docs. 

##### It must be known that the output dataframe contains Quantity elements from ~astropy.units so you should filtered out compairing to other units otherwhise it will prompt an UnitConversionError. 

## First of all the instruments and specifications will be specified

In [3]:
### Specify Instruments ###
from CtllDes.core import instrument

cam1 = instrument.Camera(0,0)
cam2 = instrument.Camera(0,1)
cam3 = instrument.Camera(0,3)


### Specify Physical Specifications ###
from CtllDes.core import specs

### useful units module, specs must have  right dimension ###
from astropy import units as u 

drag_coef = 1*u.one #dimensionless
m1 = 100*u.kg #kg
m2 = 80*u.kg #kg
m3 = 200*u.kg #kg

A = 100*u.m*u.m #m2

spec1 = specs.Specifications(m=m1,Cd=drag_coef,A=A)
spec2 = specs.Specifications(m=m1,Cd=drag_coef,A=A)
spec3 = specs.Specifications(m=m3,Cd=drag_coef,A=A)



The utility function <strong>helio_geo_synchronous</strong> receives an integer, and returns a dataframe with all the possible circular orbit classic elements that are helio and geo synchronous with a D days period. It is pretty straigthforward to transform the desired filtered row into a classical State.

In [5]:
from CtllDes.utils import sscalc as sc

D = 7
orb_df = sc.helio_rgt_synchronous(D)
orb_df

,radius,inc,N
0,11544.182918,142.248541,7.000000
1,11389.626716,138.961424,7.142857
2,11240.148578,136.071863,7.285714
3,11095.486092,133.496401,7.428571
4,10955.395204,131.177130,7.571429
...,...,...,...
66,6535.898060,96.204172,16.428571
67,6498.279346,96.079672,16.571429
68,6461.197311,95.958727,16.714286
69,6424.639809,95.841203,16.857143


Lets say that you want to filter out heights. One easy way to filter out whatever state you may want to generalize uses pandas loc. If you are not familiar with pandas, read the basic examples in their docs. 

It must be known that the output dataframe contains Quantity elements from ~astropy.units so you should filtered out compairing to other units otherwhise it will prompt an UnitConversionError. 

In [6]:
from poliastro.bodies import Earth

#this is just a sidestep to show the usability of astropy units and Bodies 
#from poliastro. 

earth_R = Earth.R_mean.to(u.km).value
print(earth_R)

min_radius = 500 + earth_R
max_radius = 900 + earth_R

filtered = orb_df.loc[(orb_df['radius'].values < max_radius) &
                      (orb_df['radius'].values > min_radius)]
filtered

6371.008400000001


,radius,inc,N
50,7222.422363,98.816646,14.142857
51,7174.189097,98.610796,14.285714
52,7126.753168,98.411801,14.428571
53,7080.093692,98.219362,14.571429
54,7034.190529,98.033200,14.714286
55,6989.024251,97.853046,14.857143
56,6944.576109,97.678649,15.000000
57,6900.828006,97.509769,15.142857


Check if you got any filtered orbits and pick. Notice that <strong>raan</strong> and <strong>nu</strong> will be set to 0. Feel free to change it to make it pass over the desired target at least one time.

In [7]:
from poliastro.twobody import Orbit #Orbit class


st1 = filtered.loc[51]
a = st1['radius']*u.km
ecc = 0 * u.one # this is cant be different than 0 since the helio_rgt is for circular orbits
inc = st1['inc']*u.deg
raan = 0 * u.deg
argp = 0 * u.deg  
nu = 0 * u.deg

orb1 = Orbit.from_classical(Earth,
                            a,
                            ecc,
                            inc,
                            raan,
                            argp,
                            nu)


Now its about time to build the satellite using the <strong>from_orbit </strong> classmethod. 

In [8]:
from CtllDes.core.satellite import Sat

In [9]:
sat1 = Sat.from_orbit(orb1,spec=spec1,instruments=cam1)

Lets keep up building the rest of the satellites with different methods. Building from a specific orbit was already covered, also adding the use of some util function that defines orbits. We could use as well for any orbit something like the following

In [10]:
### Proof of Concept ###

poc_orbit = Orbit.heliosynchronous(Earth,a=8000*u.km,ecc=0*u.one) #this classmethod is slow AF
sat_poc = Sat.from_orbit(poc_orbit) 


In [11]:
#if instruments are not defined, they will be an empty list
print(sat_poc.instruments)

#if specs are not defined, DefaultSpec will be used
print(sat_poc.spec)
print(specs.DefaultSpec)

#if status is not defined, it will be set to online
print(sat_poc.status)


[]
 m:1.0 kg, Cd:1.0, A_over_m:1.0 m2 / kg
{'m': <Quantity 1. kg>, 'Cd': <Quantity 1.>, 'A': <Quantity 1. m2>}
Online


Now, proven the ways you can build your sat, lets grab the two satellites and put them right into a TPF WalkerDelta constellation. The ways of getting synchronous or heliosynchronous orbits can be achieved by using the <strong>poliastro</strong> Orbit classmethods, and then, as showed in the example above this cell, you can build <strong>from_orbit</strong>. Imagine that we already now the classical orbit parameters for this constellation, being:


In [12]:
from poliastro.frames import Planes

p = 8000 * u.km
ecc = 0 * u.one
inc = 98 * u.deg
raan = 0 * u.rad
argp = 0 * u.rad
nu = 0 * u.rad
plane = Planes.EARTH_EQUATOR  #not necesseary to be specified

Astropy units allow you to manage input into the poliastro package without worrying to much about the unit specifically used. If you find yourself with the desired inclination in degrees, just enter <strong>u.deg</strong> instead of <strong>u.rad</strong>. Then you can directly build a Walker Delta constellation 

In [13]:
T = 2
P = 2
F = 1
constellation = ctll.Ctll.from_WalkerDelta(T,P,F,p,ecc,inc,argp)

Notice that <strong>raan</strong> and <strong>nu</strong> are not specified, but they can be, as mentioned for the other orbit mentioned before. We should specify the instruments and specifications of the satellites, as well as the TPF pattern.  

In [14]:
speks = [spec2, spec3]
instr = [cam2, cam3]

constellation = ctll.Ctll.from_WalkerDelta(T,P,F,p,ecc,inc,argp,raan_offset=40*u.deg,nu_offset=180*u.deg,specs=speks,instrumentss=instr)

After building this you may add <strong>sat1</strong> to the constellation in different ways.

In [14]:

### Creating 2 constellations and adding them ###
constellation = ctll.Ctll.from_sats([sat1]+constellation.sats)

### Using the add operator (silly example though) ###
constellation_1 = ctll.Ctll.from_sats([sat1])
constellation_2 = constellation #just for clarity
constellation = constellation_1 + constellation_2


Now that you have the constellation up and running, a propagation can be made. Let's do 10 days of simulation with an interval of 10 seconds, setting drag and J2 perturbations to true. 

In [15]:
rvs = constellation.rv(30,dt=10,J2=True,drag=True)

If you read the docs you will notice that <strong>rv</strong> method returns a list with length equal to the amount of sats, each one reporting the rv to the <strong>constellation.sats</strong> list of satellite. If you want to get the specific Ids of this satellites that are being reported you should check <strong>constellation.online_id</strong>

In [16]:
ids = constellation.online_id
ids

[UUID('72be468d-ba91-4134-bee8-208823328d0e'),
 UUID('9dbd86fc-daf1-470f-a730-49e1c8ddb32b')]

So rvs will be a list of tuples (rr,vv), each one of these being quantity objects arrays of size $\frac{T\cdot 3600 \cdot 24}{dt}$. Therefore if you want to extract and plot the orbit lets say yoy may do this by using matplotlib or using the default constellation plot, that is specially and beautifully taylor made to display awesome graphixz (sike, no that great, just cartopy default plotting). The library is data oriented, it is not self embedded the analysis of the data, that corresponds to the analytical strings that will be explained alter on on this notebook.

In [17]:
# IMPLEMENTfrom CtllDes.core.utils import plot_ctll_orbit, plot_ctll_gt
import matplotlib.pyplot as plt
%matplotlib qt5

In [18]:
last_sat_r = rvs[-1][0]

fig1 = plt.figure(figsize=(10,10))
ax = fig1.add_subplot(111, projection='3d')
ax.scatter(last_sat_r[::50,0], last_sat_r[::50,1], last_sat_r[::50,2],s=0.5,c='k') #xyz obviously

plt.show()


Very similar to this you may find the subsatellite points. In a very equal fashion, you should use the data, but for you CtllDes lovers, Ill give you some embedded crappy groundtrack plot functionality. Here you go

In [19]:
ssps = constellation.ssps(5,J2=True,drag=False)

### how you are supposed to do it, get wild again dude
first_sat_ssps = ssps[1]
first_sat_lon = first_sat_ssps[0]
first_sat_lat = first_sat_ssps[1]



fig = plt.figure()
plt.scatter(first_sat_lon,first_sat_lat,c='k',s=0.5)

### embedded
#fig2 = plot_ctll_gt(ssps)

######
plt.show()


<Quantity 0.05129254 rad>

Now that the plotting is done, your cpu thanks you. Afterwards you may want to do some nice coverage analysis. In order to do this you may have to use the Target module, since the cool methods will only recognize my special and stupid ass class called targets, just wait to see the full path to the targets folders, you will trip your tits out. (ill fix this, promise)

In [20]:
from CtllDes.targets.targets import Targets, Target

so many targets.... But there is a positive side to it, it lets you build the coverage points of sampling from country or states just like described in the cell below. 

In [ ]:
tgts = Targets.from_country('Argentina') #N is approximately 0.6 the square of the number of total sample points, more on this later
fig = tgts.plot()
plt.show()

So lets break it down. First you are building a Target container, correctly named Targets, from a country, just specify the name or ISO code. If not found and error will pop up, don't worry. So what this is actually doing, is sampling inside the territory of the specified country correcting the number of points depending on latitude. This is done in 4 simple steps.
<ul>
    <li> Check if Country exists, and found the minimum box of coordinates that surrounds it.
    <li> Generate a grid of equally N spaced points in latitude. $N\cdot \cos(latitude)$ points in longitude. This is done in order to not overestimate near the poles (imagine what would happen if you distributed uniformly on a mercator projection, forgeting about the sphericity of earth).
     <li> Filter those coordinates generated checking if they lay inside the selected country.
     </ul>
 
The procedure for states is the same. The only difference is that you have more RegEx to select the state. 

In [ ]:
tgts = Targets.from_state('Buenos Aires')
fig = tgts.plot()
plt.show()

This are a bunch of points, too much resolution. I suggest N=8 just to see the functionality.


In [23]:
tgts = Targets.from_state('Buenos Aires', N=8)
fig = tgts.plot()
plt.show()

After doing this you may want to do a coverage analysis, only if your constellation has coverage instruments on it. This is the case for this example so you could build a Coverages object from the classmethod functions. This class is actually a container for the individual Coverage object. 

You must imagine a Coverage object consisting of a target, a time of propagation, an interval dt of time of integration, and a bunch of merit figures automatically calculated, oriented to assist the design process, checking stuff like revisit time, mean gaps, max gaps, response time etc.

So:
 <ul>
    <li>Coverages == set container for Coverage (not using singular is clear enough, plz tell)
    <li> Coverage == object consisting of:
        <ul>
            <li>covs, an array with length = T*3600*24/dt containing ones or zeroes depending if the target is on sight or not.
            <li>Targets
            <li>T == Time of propagation
            <li>dt == Time interval of integration
            <li>Merit figures (see them on the dataframe example)
        </ul>
 </ul>
If you want more information on this just check Chapter 9 of O.C.D.M. from James R. Wertz. Link in bio

<p style="text-align:center">
    <a href="https://www.amazon.es/Constellation-Design-Management-Technology-Library/dp/1881883078">
        <img src="https://images-na.ssl-images-amazon.com/images/I/41Ca0XLUv6L._SX303_BO1,204,203,200_.jpg">
    </a>
</p>

In [24]:
from CtllDes.requests.coverage import Coverages

covs = Coverages.from_ctll(constellation,tgts,5,dt=5)

Satellite 1 of 2
target -62.04402324091975° -38.8197478163978°. 1 of 16
target -60.6992338529398° -38.8197478163978°. 2 of 16
target -62.04402324091975° -37.70711313209672°. 3 of 16
target -60.6992338529398° -37.70711313209672°. 4 of 16
target -59.354444464959855° -37.70711313209672°. 5 of 16
target -58.009655076979904° -37.70711313209672°. 6 of 16
target -62.04402324091975° -36.59447844779565°. 7 of 16
target -60.6992338529398° -36.59447844779565°. 8 of 16
target -59.354444464959855° -36.59447844779565°. 9 of 16
target -58.009655076979904° -36.59447844779565°. 10 of 16
target -62.04402324091975° -35.481843763494574°. 11 of 16
target -60.6992338529398° -35.481843763494574°. 12 of 16
target -59.354444464959855° -35.481843763494574°. 13 of 16
target -58.009655076979904° -35.481843763494574°. 14 of 16
target -60.6992338529398° -34.3692090791935°. 15 of 16
target -59.354444464959855° -34.3692090791935°. 16 of 16
Satellite 2 of 2
target -62.04402324091975° -38.8197478163978°. 1 of 16
target

Now you want to grab the dataframe associated to these data. Remember that there will be $ \# targets \cdot \# coverageInstruments$ points. Once you got the data frame, feel free to do some cool graphix, such as the heatmap described below, with the help of <strong>cartopy</strong> and <strong>ccrs</strong>.

In [25]:
coverage_df = covs.to_df()
coverage_df

,T,dt,Satellite ID,Target,accumulated,mean gap light,mean gap dark,response time,average time gap,max gap
0,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-62.04402324091975, -38.8197478163978)",675,135.000000,215932.500000,4863.375579,9726.000926,64820
1,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-60.6992338529398, -38.8197478163978)",1175,78.333333,107941.250000,11018.892303,22036.046875,85355
2,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-62.04402324091975, -37.70711313209672)",600,120.000000,215940.000000,4861.125058,9721.500058,64805
3,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-60.6992338529398, -37.70711313209672)",1075,107.500000,143928.333333,2580.306076,5159.863021,40560
4,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-59.354444464959855, -37.70711313209672)",2075,138.333333,107896.250000,10999.691030,21997.646412,85320
5,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-58.009655076979904, -37.70711313209672)",2525,168.333333,107873.750000,10991.014525,21980.294271,85295
6,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-62.04402324091975, -36.59447844779565)",500,100.000000,215950.000000,4859.625000,9718.500058,64795
7,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-60.6992338529398, -36.59447844779565)",1125,112.500000,143925.000000,2576.642708,5152.536690,40515
8,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-59.354444464959855, -36.59447844779565)",2000,133.333333,107900.000000,10999.273669,21996.811863,85335
9,5,5,72be468d-ba91-4134-bee8-208823328d0e,"(-58.009655076979904, -36.59447844779565)",2725,136.250000,86291.000000,6783.895023,13566.056019,44720


Now you have the coverages targets repeated #satellites times. So the user must filter out in terms of the desired output, if you want to collapse the coverages from the whole constellation, feel free to do so from the dataframe. But be aware of the fact that the merit figures dont form a linear vector space.

So if you want to really collapse and create a new type of coverage you may use Coverages builtin methods.

When I say collapse I'm refering to the consideration of "just one of something". If you collapse by satellites, you will end up with #targets merit figures, since the coverage figures are computed for the whole constellation. As I said in the first paragraph, merit figures, as accumulated time are not linear, easier to show with an example.

Imagine you got simultaneous coverage of a target at the same time from two different satellites. Ok, so now you calculate the accumulated time of coverage for each one. If you add those accumulated times, you will overestimate this merit figure, counting twice those times where the target was seen by two satellites at the same time. The correct way to do this would be to estimate if the target is in-view from a constellation perspective, regardless of the specific satellite.   

In [26]:
col_covs = covs.collapse_sats() #If no argument is specified, collapse all satellites

AttributeError: 'tuple' object has no attribute 'x'

Now with these new figures of merit that represent the whole constellation, we can play around

In [51]:
### Collapsed dataframe
col_coverage_df = col_covs.to_df()
col_coverage_df

NameError: name 'col_covs' is not defined

The following generated heatmap corresponds to the accumulated time of sight. This does not represent the percentage of coverage, but as you can imagine is very easy to compute 

32512